# Demand side flexibility


In [2]:
using Pkg
Pkg.add("CSV")
Pkg.add("DataFrames")
Pkg.add("JuMP")
Pkg.add("Gurobi")
#Pkg.add("NLopt")

    Updating registry at `C:\Users\zoele\.julia\registries\General.toml`
   Resolving package versions...
  No Changes to `C:\Users\zoele\.julia\environments\v1.8\Project.toml`
  No Changes to `C:\Users\zoele\.julia\environments\v1.8\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\zoele\.julia\environments\v1.8\Project.toml`
  No Changes to `C:\Users\zoele\.julia\environments\v1.8\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\zoele\.julia\environments\v1.8\Project.toml`
  No Changes to `C:\Users\zoele\.julia\environments\v1.8\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\zoele\.julia\environments\v1.8\Project.toml`
  No Changes to `C:\Users\zoele\.julia\environments\v1.8\Manifest.toml`


In [3]:
using CSV, DataFrames, JuMP, Gurobi

# Read the CSV data

In [4]:
SCALE_MWH = 1000
SCALE_DOLLAR = 1;

In [5]:
fixed_cost = CSV.read("fixed_cost_2.csv",DataFrame) ./ SCALE_DOLLAR

Row,Fixed Cost,Fixed OM cost
,Float64,Float64
1,543500.0,21753.1
2,6.66821e5,28779.5
3,1.1105e6,37500.0
4,3.415e6,103500.0
5,4.34537e5,0.0
6,1.86085e5,23372.3


In [6]:
variable_cost = CSV.read("variable_cost_2.csv",DataFrame) ./ SCALE_DOLLAR

Row,Variable OM,CO2 Intensity,Ramp,Start
,Float64,Float64,Float64,Float64
1,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,2.0
3,5.89,452.0,6.0,35.0
4,11.176,965.0,15.0,70.0
5,0.0,0.0,1.0,0.0
6,0.0,0.0,0.0,0.0


In [7]:
P_max_s = 95897829.43 / SCALE_MWH
P_max_w = 11158008 / SCALE_MWH;

In [8]:
Availability_matrix = CSV.read("Availability.csv", DataFrame);
print(size(Availability_matrix))

(8760, 5)

In [9]:
demand = CSV.read("demand.csv", DataFrame) ./ SCALE_MWH;
print(size(demand))
first(demand, 5)

(8760, 4)

Row,Commercial,Industrial,Residential,Transportation
,Float64,Float64,Float64,Float64
1,177.72,109.564,169.055,55.3949
2,174.776,108.33,164.425,37.6361
3,173.069,102.035,165.18,33.1084
4,171.107,96.909,168.945,29.3248
5,171.561,94.8884,173.68,23.6573


In [10]:
flexibility = CSV.read("flex.csv", DataFrame) ./ SCALE_MWH
first(flexibility, 5)

Row,Commercial,Industrial,Residential,Transportation
,Float64,Float64,Float64,Float64
1,2.34996,5.09349,15.2031,31.3589
2,2.33255,5.02394,9.44263,19.4187
3,2.34994,4.77439,5.7112,16.3404
4,2.36911,4.57863,5.11578,13.8444
5,2.44476,4.5015,4.4194,10.0628


In [11]:
flexibility_settings = CSV.read("flex_settings.csv", DataFrame)

Row,shift
,Int64
1,7
2,5
3,8
4,7


In [52]:
ramp_limits = CSV.read("ramp_limits.csv", DataFrame)
ramp_limits = [0.9, 0.05, 0.025, 0.03, 1, 1]

6-element Vector{Float64}:
 0.9
 0.05
 0.025
 0.03
 1.0
 1.0

Test without storage first

In [53]:
hrs = 1
hre = 1000
sel_hrs = hrs:hre
print(sel_hrs)
techs = 4
n_eu = 4
# pad year by 1 day
add_h = 24


C_FC = fixed_cost[1:techs,1]
C_FOM = fixed_cost[1:techs,2]
C_VOM = variable_cost[1:techs,1]
E_CO2 = variable_cost[1:techs,2]
C_ramp = variable_cost[1:techs,3]
C_start = variable_cost[1:techs,3]

D_b = demand[sel_hrs, 1:4] # Baseline demand with no flex
print(size(D_b))
D_b = vcat(reverse(reverse(demand)[1:add_h, 1:n_eu]), D_b, D_b[1:24, 1:n_eu])
print(size(D_b))
total_demand = sum(eachcol(D_b))
X_D = total_demand

A_f = flexibility[sel_hrs, 1:4]
A_f = vcat(reverse(reverse(flexibility)[1:add_h, 1:n_eu]), A_f, A_f[1:24, 1:n_eu])
H_shift = [7, 5, 8, 7]

A_r = ramp_limits[1:techs, 1]

A_s = Availability_matrix[sel_hrs,1:techs]
A_s = vcat(reverse(reverse(Availability_matrix)[1:add_h, 1:n_eu]), A_s, A_s[1:24, 1:n_eu])
print(size(A_s))
# over time, opt each year with worst day for availability and highest demand?

1:1000(1000, 4)(1048, 4)(1048, 4)

# Constants

In [54]:
eta = 0.8; # Efficiency of the battery
CO2_price = 100; # price of the CO2 per ton

# Model

In [55]:
model = Model(Gurobi.Optimizer)
set_optimizer_attribute(model, "NonConvex", 2)
# set_optimizer_attribute(model, "IterationLimit", 10000)
println("Model: initialized!\n")

n = size(C_FC, 1) # number of technologies
println("Number of technologies: ", n)
T = size(D_b, 1) # hours in simulation
println("Timespan: ", T)
time_ls = 1+add_h:T-add_h

# Non-negativity constraints
# The installed power capacity of the mix
@variable(model, P[1:n] >= 0)

# The hourly power generation of each technology
@variable(model, X_gen[1:T, 1:n] >= 0); # hourly production/use for each technology

###### CONSTRAINTS ######

# Demand constraint - meet demand per hour
println("Demand constraint")
@constraint(model, Con_dh[t = 1:T], sum(X_gen[t, i] for i in 1:n) == X_D[t]); # TODO: add curtailment
# Meet total demand
@constraint(model, Con_d, sum(X_gen[t, i] for i in 1:n for t in 1:T) == sum(total_demand));

# Installed capacity renewables < max capacity
println("Renewable maximum installed capacity constraint")
@constraint(model, Con_sgen[t = 1:T], X_gen[t, 1] <= P_max_s);
@constraint(model, Con_wgen[t = 1:T], X_gen[t, 2] <= P_max_w);

# Cannot produce more than the installed power
println("Installed power constraint")
@constraint(model, Con_p[i = 1:n, t = 1:T], X_gen[t, i] <= P[i])

# Cannot produce less than the minimum stable generation: X_gen must be zero or min to max
# min_usage = [0, 0.1, 0, 0] #TODO - cant be more than ramp
# @constraint(model, [i = 1:n, t = 1:T], X_gen[t, i] >= P[i]*min_usage[i])

# For each tech, at each hour, usage cannot exceed available capacity
println("Availability constraint")
@constraint(model, Con_a[i = 1:n, t = 1:T], X_gen[t, i] <= P[i]*A_s[t, i]) 

# For each tech, at each hour, change in production cannot exceed ramp rate (TODO might need to convert this to time)
println("Ramp constraint")
# Auxiliary variables to linearize the rate of change constraint
# @variable(model, dx[time_ls, 1:n] >= 0);
# @constraint(model, [i = 1:n, t = time_ls-1], (X_gen[t+1, i] - X_gen[t, i]) == dx[t, i]) 
# @constraint(model, [i = 1:n, t = time_ls], dx[t, i] <= A_r[i]*X_gen[t, i]) 
# @constraint(model, [i = 1:n, t = time_ls-1], dx[t, i] >= -A_r[i]*X_gen[t, i]) 

# @constraint(model, Con_rdwn[i = 1:n, t = time_ls], -A_r[i]*X_gen[t, i] <= (X_gen[limit_bound_max(t+1, T), i] - X_gen[t, i]) <= A_r[i]*X_gen[t, i]) 

@constraint(model, Con_rdwn[i = 1:n, t = time_ls], (X_gen[t+1, i] - X_gen[t, i]) >= -A_r[i]*X_gen[t, i]) 
@constraint(model, Con_rup[i = 1:n, t = time_ls], (X_gen[t+1, i] - X_gen[t, i]) <= A_r[i]*X_gen[t, i]);

Set parameter Username

--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only - expires 2023-12-11
Set parameter NonConvex to value 2
Model: initialized!

Number of technologies: 4
Timespan: 1048


Demand constraint
Renewable maximum installed capacity constraint
Installed power constraint
Availability constraint
Ramp constraint


## Objective

In [56]:
# # Add objective function: minimize the total costs TODO: discount rates
# function start_cost(x_0, x_1, i)
#     if x_0 == 0
#         C_start[i]
#     else
#         0
#     end
# end

# @variable(model, X_roc[time_ls, 1:n])
# @constraint(model, [i = 1:4, t = time_ls], X_roc[t, i] == X_gen[t, i]-X_gen[limit_bound_max(t+1, T), i])

#TODO RAMP AND STARTUP COSTS
# @objective(model, Min, 
#     (
#         sum((C_FC[i] + C_FOM[i])*P[i] for i in 1:n) +
#         sum(sum((C_VOM[i] + CO2_price * E_CO2[i])* X_gen[t, i] for i in 1:n) for t in time_ls) +
#         sum(sum(abs(X_roc)*C_ramp[i] for i in 1:n) for t in 2:T) 
# ));

@objective(model, Min, 
    (
        sum((C_FC[i] + C_FOM[i])*P[i] for i in 1:n) +
        sum(sum((C_VOM[i] + CO2_price * E_CO2[i])* X_gen[t, i] for i in 1:n) for t in time_ls)
));

In [57]:
#Optimize
optimize!(model)

Set parameter NonConvex to value 2
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: AMD Ryzen 9 5900HS with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 19529 rows, 4196 columns and 42780 nonzeros
Model fingerprint: 0x344bf9e5
Coefficient statistics:
  Matrix range     [1e-08, 2e+00]
  Objective range  [5e+04, 4e+06]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+02, 7e+05]
Presolve removed 9244 rows and 2026 columns
Presolve time: 0.06s
Presolved: 10285 rows, 2170 columns, 25642 nonzeros

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Ordering time: 0.00s

Barrier statistics:
 Dense cols : 3
 AA' NZ     : 8.450e+04
 Factor NZ  : 1.479e+05 (roughly 6 MB of memory)
 Factor Ops : 2.239e+06 (less than 1 second per iteration)
 Threads    : 6

                  Objective                Residual
Iter       Prim

## Results

In [58]:
value.(P)

LoadError: Result index of attribute MathOptInterface.VariablePrimal(1) out of bounds. There are currently 0 solution(s) in the model.

In [ ]:
X_gen_matrix = value.(X_gen);
    X_gen_matrix = transpose(X_gen_matrix)
size(X_gen_matrix)

(4, 1048)

In [ ]:
using Plots

@userplot StackedArea

# a simple "recipe" for Plots.jl to get stacked area plots
# usage: stackedarea(xvector, datamatrix, plotsoptions)
@recipe function f(pc::StackedArea)
    x, y = pc.args
    n = length(x)
    y = cumsum(y, dims=2)
    seriestype := :shape

    # create a filled polygon for each item
    for c=1:size(y,2)
        sx = vcat(x, reverse(x))
        sy = vcat(y[:,c], c==1 ? zeros(n) : reverse(y[:,c-1]))
        @series (sx, sy)
    end
end

a = X_gen_matrix[1, time_ls]
b = X_gen_matrix[2, time_ls]
c = X_gen_matrix[3, time_ls]
d = X_gen_matrix[4, time_ls]
x = time_ls

plotly()
stackedarea(x, [d c b a], labels=["Coal" "CCGT" "Wind" "Solar"], lw=0)
plot!(x, total_demand[time_ls], label="Demand", lc=:black, lw=1, ls=:dot)
# plot!(x, value.(X_D)[:], label="Demand, shifted", lc=:black, lw=2)
plot!(size=(1400,400))